In [1]:
import numpy as np
import pandas as pd
import cv2

import pickle

import utlis

In [19]:
# load the model from disk
pitch_model_path = 'Saved models/model_pitch.pkl'
roll_model_path = 'Saved models/model_roll.pkl'
yaw_model_path = 'Saved models/model_yaw.pkl'

pitch_model = pickle.load(open(pitch_model_path, 'rb'))
roll_model = pickle.load(open(roll_model_path, 'rb'))
yaw_model = pickle.load(open(yaw_model_path, 'rb'))

In [23]:
cap = cv2.VideoCapture(0)

while True:
    # read a frame
    _, img = cap.read()
    
    # flip image 
    img = cv2.flip(img, 1)
    
    # Convert BGR to RGB 
    img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Get image dimensions
    img_h, img_w, img_c = img_RGB.shape
    
    img, landmarks = utlis.find_face_landmarks(img, 3)

    if not landmarks.empty:
        # Get nose and chin points
        n_x = landmarks['x2']
        n_y = landmarks['y2']
        chin_x = landmarks['x429']
        chin_y = landmarks['y429']

        # subtract nose point from all points and divide by the distance between nose and the chin
        lands_normalized = utlis.normalize_df(landmarks, n_x, n_y, chin_x, chin_y)

        # predict pitch, roll, yaw
        pitchs = pitch_model.predict(lands_normalized)
        rolls = roll_model.predict(lands_normalized)
        yaws = yaw_model.predict(lands_normalized)

        for i, (pitch, roll, yaw) in enumerate(zip(pitchs, rolls, yaws)):
            img = utlis.draw_axis(img, yaw, pitch, roll, int(n_x[i]*img_w), int(n_y[i]*img_h))

    cv2.imshow("window", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()